In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1
)

### from_tmeplate() 메소드
- 변수를 중괄호로 묶어서 템플릿에 정의

In [2]:
from langchain_core.prompts import PromptTemplate

# template 정의, {}안의 내용은 이후에 값이 들어갈 자리
template = "{name}의 직업은 무엇인가요?"

# from_template 메소드를 사용해 PromptTemplate 객체를 생성
prompt = PromptTemplate.from_template(template)

print(prompt)

input_variables=['name'] input_types={} partial_variables={} template='{name}의 직업은 무엇인가요?'


In [3]:
# prompt 생성 (완성) format() 메소드를 이용해 변수에 값을 넣음
prompt = prompt.format(name="bear")

print(prompt)

bear의 직업은 무엇인가요?


In [18]:
from langchain_core.prompts import PromptTemplate

template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

print(prompt)

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


In [19]:
# 기본적으로는 딕셔너리 형태로 변수를 작성해야 하지만
# 변수가 한 개일 때는 변수만으로 작성이 가능

# print(chain.invoke({"language" : "Python"}).content)
print(chain.invoke("Python").content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1989년에 개발되었습니다. 그는 이 언어를 만들 때, 코드의 가독성과 사용의 용이성을 중시하였으며, 1991년에 첫 번째 버전을 공개했습니다. 이후 Python은 전 세계적으로 널리 사용되는 프로그래밍 언어로 성장하였습니다.


### PromptTemplate 객체 생성과 동시에 prompt 생성
- input_variables 인자를 사용해 변수를 지정한다.
- 템플릿에 작성한 변수가 input_variables에 없으면 예외를 발생시켜준다.

In [9]:
# template 정의
template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["language"],
)

print(prompt)

prompt.format(language="Python")

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


'Python는 누가 만들었나요?'

**partial_variables**
- 연산 중 미리 계산된 변수를 프롬프트 템플릿에 지정해 넣을 수 있다.
- 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우 사용  
ex) 날짜, 시간

In [12]:
# template 정의
template = "{language1}과 {language2}는 각각 누가 만들었나요?"

# PromptTemplate 객체 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["language1"],
    partial_variables={
        "language2" : "java" # dictionary 형태로 partial_variables 작성
    }
)

print(prompt)

input_variables=['language1'] input_types={} partial_variables={'language2': 'java'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [13]:
prompt.format(language1="Python")

'Python과 java는 각각 누가 만들었나요?'

**partial()**
- 연산 중에 미리 계산된 변수를 프롬프트 템플릿에 지정해서 넣을 수 있다.

In [14]:
prompt_partial = prompt.partial(language2="Javascript")

print(prompt_partial)

input_variables=['language1'] input_types={} partial_variables={'language2': 'Javascript'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [15]:
prompt_partial.invoke("Python")

StringPromptValue(text='Python과 Javascript는 각각 누가 만들었나요?')

In [20]:
chain = prompt_partial | llm

print(chain.invoke("Python").content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 이 언어를 만들 때 코드의 가독성과 사용의 용이성을 중시했습니다.

JavaScript는 브렌던 아이크(Brendan Eich)에 의해 1995년에 개발되었습니다. 그는 넷스케이프(Netscape)에서 일할 당시 웹 브라우저에서 동적인 기능을 추가하기 위해 JavaScript를 만들었습니다.

두 언어 모두 현재 매우 널리 사용되고 있으며, 각각의 분야에서 중요한 역할을 하고 있습니다.


### 파일로부터 template 읽어오기

- prompt를 편하게 작성 및 수정
- 상황에 맞게 파일을 작성해두면 필요할 때마다 꺼내서 쓸 수 있다.

In [21]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/language_simple.yml", encoding="utf-8")

print(prompt)

input_variables=['language'] input_types={} partial_variables={} template='{language} 언어에 대해 3줄로 설명해줘'


In [22]:
prompt.format(language="Python")

'Python 언어에 대해 3줄로 설명해줘'

In [23]:
prompt2 = load_prompt("prompts/language.yml", encoding="utf-8")

print(prompt2.format(language="Python"))

Python 언어에 대해 설명해주세요.
언어의 특징을 다음의 양식에 맞게 정리하세요.
300자 내외로 작성하세요.
한글로 작성하세요.
---
#양식
1. 특징
2. 제작자
3. 대표적인 프레임워크
4. 많이 사용되는 분야



In [27]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt2 | ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0) | StrOutputParser()

answer = chain.invoke("Python")

print(answer)

1. 특징: Python은 간결하고 가독성이 높은 문법을 가진 고급 프로그래밍 언어로, 다양한 프로그래밍 패러다임(객체지향, 함수형 등)을 지원합니다. 동적 타이핑과 자동 메모리 관리를 제공하여 개발 속도를 높이고, 다양한 라이브러리와 패키지를 통해 확장성이 뛰어납니다.

2. 제작자: Python은 귀도 반 로섬(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다.

3. 대표적인 프레임워크: Django, Flask, FastAPI 등이 있으며, 웹 개발에 많이 사용됩니다.

4. 많이 사용되는 분야: 데이터 과학, 인공지능, 웹 개발, 자동화 스크립트, 게임 개발 등 다양한 분야에서 널리 사용됩니다.


### ChatPromptTemplate

- 대화 목록을 프롬프트로 주입하고자 할 때 사용할 수 있다.
- 메시지는 튜플 형태로 전달
    -("role", "message") 구성되고, 리스트 형태로 생성 가능

**role**
- system : 시스템 설정 메시지로 주로 전역 설정을 할 때 사용
- human : 사용자의 입력 메세지
- ai : AI의 답변 메세지

In [29]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{language}의 제작자는 누구인가요?")

print(chat_prompt)

input_variables=['language'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='{language}의 제작자는 누구인가요?'), additional_kwargs={})]


In [30]:
print(chat_prompt.format(language="Python"))

Human: Python의 제작자는 누구인가요?


In [31]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 침절한 인공지능 어시스턴트입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}")
    ]
)

messages = chat_template.format_messages(
    name="구찌", user_input="당신의 이름은 무엇입니까?"
)

print(messages)

[SystemMessage(content='당신은 침절한 인공지능 어시스턴트입니다. 당신의 이름은 구찌입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]


In [32]:
print(llm.invoke(messages).content)

제 이름은 구찌입니다! 당신은 어떤 도움이 필요하신가요?


### MessagePlaceHolder

- 아직 확정된 메세지가 아니지만, 나중에 채워질 메세지 위치를 잡아두기 위해 사용한다.
- 보통 대화 기록을 하고 싶을 때 사용한다.

In [50]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다."
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다.")
    ]
)

print(chat_prompt)

input_variables=['conversation', 'word_count'] input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.m

In [51]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?
AI: 아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다.
Human: set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ
AI: 네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [52]:
chain = chat_prompt | llm | StrOutputParser()

In [53]:
input = {
    "word_count":5,
    "conversation":[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
}

chain.invoke(input)

'파이썬, 리스트, 중복 제거, set.'

In [55]:
!pip install langchain-community

In [60]:
from langchain_community.llms import Ollama

# model
llm = Ollama(model="gemma2:9b")

response = llm.invoke("지구의 자전 주기는?")

response

C:\Users\20113\AppData\Local\Temp\ipykernel_18272\1576383684.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma2:9b")


' 지구의 자전 주기는 약 24시간입니다. 이는 하루라고도 합니다.\n\n지구가 한 바퀴 돌아서 태양에 대해 다시 같은 위치로 돌아오는 시간은 시내에서 18시 37분 56초 입니다.\n'

In [57]:
from langchain.chat_models import ChatOllama

model = ChatOllama(
    model="gemma2:9b",
    temperature=0.1
)



In [58]:
chain = chat_prompt | model | StrOutputParser()

chain.invoke(input)

'파이썬, 중복 제거, set 사용, 간단 해결, 유용  \n'